## 단어사전에 넣은 키워드 추출

- 고유명사를 뽑기 위한 작업
- ex) [나는 취업박람회에 가서 직업체험을 경험했다.] --> 취업박람회(취업+박람회), 직업체험(직업+체험)

In [1]:
from konlpy.tag import Mecab

import pandas as pd
import numpy as np
import os

In [2]:
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [3]:
content = pd.read_csv('using_data/policy_content.csv', encoding='cp949')

content['content_all'] = content['name'] + ' ' + content['description'] + ' ' + content['content'] # 이름,내용,지원내용 한번에 처리

In [9]:
# 문장 띄어쓰기로 나눠서 명사 추출 - (고유명사 뽑기 위해)

def pos_function(text):
    content_split = []

    for i in text.split():
        content_split.append(mecab.pos(i))

    new_words_list = []
    noun_pos = ['NNG','XSN','NNP']    # 일반명사, 명사 파생 접미사, 고유명사인 애들만 붙이기 
     
    for i in content_split:
        word = ''
        for j in i:
            if j[1] in noun_pos:
                word = word + j[0]
            else:
                break
        new_words_list.append(word)
    
    # 비교용 nouns
    n = mecab.nouns(text)
    
    # 이미 dictionary에 있는 단어 제외하기 
    # 수정 필요 - 한번에 remove가 안돼서 4번 반복
    for i in new_words_list:
        if i in n:
            new_words_list.remove(i)
    for i in new_words_list:
        if i in n:
            new_words_list.remove(i)
    for i in new_words_list:
        if i in n:
            new_words_list.remove(i)                
    for i in new_words_list:
        if i in n:
            new_words_list.remove(i) 
    
    # 공백인 단어 제거
    while '' in new_words_list:            
        if '' in new_words_list:
            new_words_list.remove('')
    
    return new_words_list

In [ ]:
#추출한 명사 list

word_list = []

for i in range(0,1000):
    w = pos_function(content['content_all'][i])
    word_list.extend(w)
    
word_list

In [ ]:
#중복제거

word_list
word_unique = list(set(word_list))

In [12]:
# 과 -> 제거, 의/들/이 -> 빼기

##### 아예 제거

search = "과"
for word in word_unique:
    if word[-1] == search:
        word_unique.remove(word)
        
word_unique.append('관련학과')
word_unique.append('특성화학과')
word_unique.append('스마트제조융합학과')

search = "인"
for word in word_unique:
    if word[-1] == search:
        word_unique.remove(word)

word_unique.extend(['통합적','핵심분야','유망산업','긍정적','심층적','점진적'])

search = "와"
for word in word_unique:
    if word[-1] == search:
        word_unique.remove(word)

search = "이"
for word in word_unique:
    if word[-1] == search:
        word_unique.remove(word)
        
word_unique.extend(['안전지킴이','매칭데이','공감페이','넥타이','자찬릴레이','멘토데이','면접관','일구데이','오디션데이','외톨이','과학기술일자리진흥원'])
        
##### 한 글자 빼기

search = '의'
for i, word in enumerate(word_unique):
    if search in word: 
        word_unique[i] = word.rstrip(search)

search = '들'
for i, word in enumerate(word_unique):
    if search in word: 
        word_unique[i] = word.rstrip(search)

search = '별'
for i, word in enumerate(word_unique):
    if search in word: 
        word_unique[i] = word.rstrip(search)

In [13]:
# 2글자 이하 단어 삭제

results = []

for i in word_unique:
    if len(i) >= 3:
        results.append(i)

In [14]:
# 마지막 중복제거

results = list(set(results))

In [15]:
df = pd.DataFrame({'words': results})
df.to_csv('word_dictionary_final.csv', encoding = 'euc-kr', index = False)

In [ ]:
df # 확인 후 필요없는 단어 삭제

## 단어사전 생성

In [ ]:
# 현재 디렉토리 확인

os.getcwd()

In [5]:
# 사전에 넣기 위해 mecab으로 이동

os.chdir('C:\mecab')

In [ ]:
os.getcwd()

In [ ]:
ls

In [ ]:
# 단어사전 불러오기
words = pd.read_csv('C:/Users/juwan/Desktop/산학프로젝트/using_data/word_dictionary_final.csv', encoding='euc-kr')
words

In [ ]:
# 명사 목록 list로 추출
word_lst = words['words']
word_lst

In [10]:
# 등록할 단어 목록 list
word_list = word_lst

In [11]:
# 종성여부를 판단하기 위한 라이브러리 설치와 판단하는 함수
#jamo 라이브러리 설치
# !pip install jamo

In [12]:
#종성 여부를 판단하는 함수
from jamo import h2j, j2hcj


def get_jongsung_TF(sample_text):
    sample_text_list = list(sample_text)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]

    jongsung_TF = "T"

    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
        jongsung_TF = "F"

    return jongsung_TF

In [13]:
# nnp.csv에 단어 추가

with open("C:/Mecab/user-dic/nnp.csv", 'r', encoding='euc-kr') as f:
    file_data = f.readlines()

word_list = word_lst

for word in word_list:
    jongsung_TF = get_jongsung_TF(word)

    line = '{},,,,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word)

    file_data.append(line)

In [14]:
with open("./user-dic/nnp.csv", 'w', encoding='utf-8') as f: 
    for line in file_data: 
        f.write(line)

In [ ]:
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f: 
    file_new = f.readlines() 
file_new

# 여기서 주피터 끄고 확인

In [ ]:
# 컴파일 후 확인 (tools\add-userdic-win.ps1)

content = pd.read_csv('using_data/policy_content.csv', encoding='cp949')

In [10]:
from konlpy.tag import Mecab

In [ ]:
# 잘 등록되었나 확인

text = content['description'][36]

print(text)
print()
print(mecab.nouns(text))

In [ ]:
mecab.nouns('플랫폼')

In [ ]:
mecab.nouns('나는 청춘센터에 가서 일한다.')

## 단어사전 우선순위 변경

In [14]:
with open('C:/Mecab/mecab-ko-dic/user-nnp.csv', 'r', encoding = 'utf-8') as f:
    user_nnp = f.readlines()

temp = []
for i in user_nnp:
    temp.append(i.split(','))

# 4번째 열의 숫자가 단어의 가중치 - 낮을수록 우선순위가 높음(반비례)
user_nnp = []    
for i in temp:
    i[3] = '0'
    user_nnp.append(','.join(i))

with open('C:/Mecab/mecab-ko-dic/user-nnp.csv', 'w', encoding = 'utf-8') as f:
    for line in user_nnp: 
        f.write(line)

# 주피터 끄고 컴파일 진행 (tools\compile-win.ps1)

In [3]:
print(mecab.nouns('취업준비생인데 일자리가 급해서요. 괜찮은 중소기업이라도 있을까요?'))
print(mecab.nouns('취준생인데 일자리가 급해서요. 괜찮은 중소기업이라도 있을까요?'))
print(mecab.nouns('취준중인데요. 일자리가 급해서요. 괜찮은 중소기업이라도 있을까요?'))

['취업준비생', '일자리', '중소기업']
['취준생', '일자리', '중소기업']
['취준', '중', '일자리', '중소기업']


In [4]:
print(mecab.nouns('자기소개서 어떻게 쓰나요'))
print(mecab.nouns('취업성공패키지 신청하고 싶어요'))

['자기소개서']
['취업성공패키지', '신청']
